In [1]:
import json
from collections import Counter

def check_coco_id_uniqueness(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)
    
    image_ids = [img['id'] for img in data['images']]
    annotation_ids = [ann['id'] for ann in data['annotations']]
    
    image_id_counts = Counter(image_ids)
    annotation_id_counts = Counter(annotation_ids)
    
    duplicate_image_ids = [item for item, count in image_id_counts.items() if count > 1]
    duplicate_annotation_ids = [item for item, count in annotation_id_counts.items() if count > 1]
    
    if len(duplicate_image_ids) == 0:
        print("All image IDs are unique.")
    else:
        print(f"Warning: Duplicate image IDs found: {duplicate_image_ids}")
        
    if len(duplicate_annotation_ids) == 0:
        print("All annotation IDs are unique.")
    else:
        print(f"Warning: Duplicate annotation IDs found: {duplicate_annotation_ids}")
        # Find the images containing the non-unique annotation ids
        non_unique_ann_image_ids = []
        for ann_id in duplicate_annotation_ids:
            image_ids = [ann['image_id'] for ann in data['annotations'] if ann['id'] == ann_id]
            non_unique_ann_image_ids.extend(image_ids)
        non_unique_ann_image_ids = list(set(non_unique_ann_image_ids))
        print(f"Images containing non-unique annotation ids: {non_unique_ann_image_ids}")
        
    return data

# Example usage
annotation_file = 'Dataset/annotations/instances_val2017.json'
data = check_coco_id_uniqueness(annotation_file)

# Example usage
annotation_file = 'Dataset/annotations/instances_train2017.json'
data = check_coco_id_uniqueness(annotation_file)

All image IDs are unique.
All annotation IDs are unique.
All image IDs are unique.
All annotation IDs are unique.


In [7]:
json_path = 'Dataset/annotations/instances_val2017.json'
json = json.load(open(json_path, 'r'))
standard_category = json['categories']

In [8]:
print(data.keys())
print(standard_category)

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
[{'supercategory': 'landmark', 'id': 3, 'name': 'crater'}]


# Adjust BBoxes:

In [9]:
# If the point is outside the image, it is adjusted:
def adjust_bbox_annotation(annotation):
    def convert_num(num):
        if num <=0:
            return 0
        elif num >= 511:
            return 512
        else:
            return int(num)
        
    bbox = annotation['bbox']
    new_bbox = [convert_num(x) for x in bbox]
    x_min, y_min, width, height = new_bbox
    # [x_min, y_min, width, height]
    
    destra = x_min + width
    sotto = y_min + height
    
    if destra >= 512:
        width = 512 - x_min
    
    if sotto >= 512:
        height = 512 - y_min

    destra = x_min + width
    sotto = y_min + height
    
    assert destra <= 512, f'ERRORE A DESTRA: {destra}'
    assert sotto <= 512, f'ERRORE SOTTO: {sotto}'
    
    annotation['bbox'] = new_bbox
    return annotation


adjusted_annotations_list = [adjust_bbox_annotation(x) for x in data['annotations']]
data['annotations'] = adjusted_annotations_list

In [10]:
# adjust the data file, iscrowd = 0
for ann in data['annotations']:
    ann['iscrowd'] = 0
    
# adjust the area of the annotations:
for ann in data['annotations']:
    ann['area'] = ann['bbox'][2] * ann['bbox'][3]
    
data['categories'] = standard_category

In [6]:
import json
# save the new json file:
with open('RevAnnots/instances_val2017.json', 'w') as f:
    json.dump(data, f)

# Adjust Annotations IDs

In [ ]:
image_ids = [img['id'] for img in data['images']]
annotation_ids = [ann['id'] for ann in data['annotations']]

In [ ]:
# Step 2: Update annotation IDs
for i, annotation in enumerate(data['annotations']):
    annotation['id'] = i + 1  # Making IDs start from 1

# Step 3: Save the modified JSON data
with open('/home/roberto/PythonProjects/S2RAWVessel/mmdetection/data/S2ESA/sen2ships/sen2ships/coco_annotations_rev_RDP.json', 'w') as f:
    json.dump(data, f, indent=4)